<a href="https://colab.research.google.com/github/2Swon/DeepLearing/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms, datasets

# hyper parameter 설정

In [ ]:
from torchvision import transforms, datasets

## 트레이닝 필요한 파라메터를 설정하기
lr = 1e-3
batch_size = 64
num_epoch = 10

ckpt_dir = './checkpoint'
log_dir = './log'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 구현

In [ ]:
## 네트워크를 구축하기
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, stride=1, padding=0, bias=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=1, padding=0, bias=True)
        self.drop2 = nn.Dropout2d(p=0.5)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.relu2 = nn.ReLU()

        self.fc1 = nn.Linear(in_features=320, out_features=50, bias=True)
        self.relu1_fc1 = nn.ReLU()
        self.drop1_fc1 = nn.Dropout2d(p=0.5)

        self.fc2 = nn.Linear(in_features=50, out_features=10, bias=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.drop2(x)
        x = self.pool2(x)
        x = self.relu2(x)

        x = x.view(-1, 320)

        x = self.fc1(x)
        x = self.relu1_fc1(x)
        x = self.drop1_fc1(x)

        x = self.fc2(x)

        return x

In [ ]:
## 네트워크를 저장하거나 불러오는 함수 작성하기
def save(ckpt_dir, net, optim, epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},
               './%s/model_epoch%d.pth' % (ckpt_dir, epoch))

def load(ckpt_dir, net, optim):
    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort()

    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])

    return net, optim


# 데이터 불러오기

In [ ]:
## MNIST 데이터 불러오기
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])

dataset = datasets.MNIST(download=True, root='./', train=True, transform=transform)
loader = DataLoader(dataset, batch_size=batch_size,shuffle=True, num_workers=0)

num_data = len(loader.dataset)
num_batch = np.ceil(num_data / batch_size)

## 네트워크 설정 및 필요한 손실함수 구현하기
net = Net().to(device)
params = net.parameters()

fn_loss = nn.CrossEntropyLoss().to(device)
fn_pred = lambda output: torch.softmax(output, dim=1)
fn_acc = lambda pred, label: ((pred.max(dim=1)[1] == label).type(torch.float)).mean()

optim = torch.optim.Adam(params, lr=lr)

writer = SummaryWriter(log_dir=log_dir)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



# training

In [ ]:
## 트레이닝 시작하기
for epoch in range(1, num_epoch + 1):
    net.train()

    loss_arr = []
    acc_arr = []

    for  batch, (input, label) in enumerate(loader, 1):
        input = input.to(device)
        label = label.to(device)

        output = net(input)
        pred = fn_pred(output)

        optim.zero_grad()

        loss = fn_loss(output, label)
        acc = fn_acc(pred, label)

        loss.backward()

        optim.step()

        loss_arr += [loss.item()]
        acc_arr += [acc.item()]

        print('TRAIN: EPOCH %04d/%04d | BATCH %04d/%04d | LOSS: %.4f | ACC %.4f' %
              (epoch, num_epoch, batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))

    writer.add_scalar('loss', np.mean(loss_arr), epoch)
    writer.add_scalar('acc', np.mean(acc_arr), epoch)

    save(ckpt_dir = ckpt_dir, net=net, optim=optim, epoch=epoch)

writer.close()

/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


TRAIN: EPOCH 0009/0010 | BATCH 0447/0938 | LOSS: 0.1426 | ACC 0.9580
TRAIN: EPOCH 0009/0010 | BATCH 0448/0938 | LOSS: 0.1429 | ACC 0.9580
TRAIN: EPOCH 0009/0010 | BATCH 0449/0938 | LOSS: 0.1432 | ACC 0.9579
TRAIN: EPOCH 0009/0010 | BATCH 0450/0938 | LOSS: 0.1431 | ACC 0.9579
TRAIN: EPOCH 0009/0010 | BATCH 0451/0938 | LOSS: 0.1430 | ACC 0.9579
TRAIN: EPOCH 0009/0010 | BATCH 0452/0938 | LOSS: 0.1430 | ACC 0.9580
TRAIN: EPOCH 0009/0010 | BATCH 0453/0938 | LOSS: 0.1429 | ACC 0.9580
TRAIN: EPOCH 0009/0010 | BATCH 0454/0938 | LOSS: 0.1430 | ACC 0.9579
TRAIN: EPOCH 0009/0010 | BATCH 0455/0938 | LOSS: 0.1429 | ACC 0.9580
TRAIN: EPOCH 0009/0010 | BATCH 0456/0938 | LOSS: 0.1427 | ACC 0.9581
TRAIN: EPOCH 0009/0010 | BATCH 0457/0938 | LOSS: 0.1425 | ACC 0.9582
TRAIN: EPOCH 0009/0010 | BATCH 0458/0938 | LOSS: 0.1424 | ACC 0.9582
TRAIN: EPOCH 0009/0010 | BATCH 0459/0938 | LOSS: 0.1424 | ACC 0.9583
TRAIN: EPOCH 0009/0010 | BATCH 0460/0938 | LOSS: 0.1422 | ACC 0.9584
TRAIN: EPOCH 0009/0010 | BATCH 046

# validation

In [ ]:
lr = 1e-3
batch_size = 64
num_epoch = 10

ckpt_dir = './checkpoint'
log_dir = './log'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 네트워크를 구축하기
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5, stride=1, padding=0, bias=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5, stride=1, padding=0, bias=True)
        self.drop2 = nn.Dropout2d(p=0.5)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.relu2 = nn.ReLU()

        self.fc1 = nn.Linear(in_features=320, out_features=50, bias=True)
        self.relu1_fc1 = nn.ReLU()
        self.drop1_fc1 = nn.Dropout2d(p=0.5)

        self.fc2 = nn.Linear(in_features=50, out_features=10, bias=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.drop2(x)
        x = self.pool2(x)
        x = self.relu2(x)

        x = x.view(-1, 320)

        x = self.fc1(x)
        x = self.relu1_fc1(x)
        x = self.drop1_fc1(x)

        x = self.fc2(x)

        return x

## 네트워크를 저장하거나 불러오는 함수 작성하기
def save(ckpt_dir, net, optim, epoch):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    torch.save({'net': net.state_dict(), 'optim': optim.state_dict()},
               './%s/model_epoch%d.pth' % (ckpt_dir, epoch))

def load(ckpt_dir, net, optim):
    ckpt_lst = os.listdir(ckpt_dir)
    ckpt_lst.sort()

    dict_model = torch.load('./%s/%s' % (ckpt_dir, ckpt_lst[-1]))

    net.load_state_dict(dict_model['net'])
    optim.load_state_dict(dict_model['optim'])

    return net, optim

## MNIST 데이터 불러오기
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])

dataset = datasets.MNIST(download=True, root='./', train=False, transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)

num_data = len(loader.dataset)
num_batch = np.ceil(num_data / batch_size)

## 네트워크 설정 및 필요한 손실함수 구현하기
net = Net().to(device)
params = net.parameters()

fn_loss = nn.CrossEntropyLoss().to(device)
fn_pred = lambda output: torch.softmax(output, dim=1)
fn_acc = lambda pred, label: ((pred.max(dim=1)[1] == label).type(torch.float)).mean()

optim = torch.optim.Adam(params, lr=lr)

writer = SummaryWriter(log_dir=log_dir)

net, optim = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

## 트레이닝 시작하기
with torch.no_grad():
    # net.train()
    net.eval()

    loss_arr = []
    acc_arr = []

    for batch, (input, label) in enumerate(loader, 1):
        input = input.to(device)
        label = label.to(device)

        output = net(input)
        pred = fn_pred(output)

        loss = fn_loss(output, label)
        acc = fn_acc(pred, label)

        loss_arr += [loss.item()]
        acc_arr += [acc.item()]

        print('TEST: BATCH %04d/%04d | LOSS: %.4f | ACC %.4f' %
              (batch, num_batch, np.mean(loss_arr), np.mean(acc_arr)))

TEST: BATCH 0001/0157 | LOSS: 0.0030 | ACC 1.0000
TEST: BATCH 0002/0157 | LOSS: 0.0068 | ACC 1.0000
TEST: BATCH 0003/0157 | LOSS: 0.0067 | ACC 1.0000
TEST: BATCH 0004/0157 | LOSS: 0.0066 | ACC 1.0000
TEST: BATCH 0005/0157 | LOSS: 0.0108 | ACC 0.9969
TEST: BATCH 0006/0157 | LOSS: 0.0142 | ACC 0.9948
TEST: BATCH 0007/0157 | LOSS: 0.0177 | ACC 0.9933
TEST: BATCH 0008/0157 | LOSS: 0.0240 | ACC 0.9922
TEST: BATCH 0009/0157 | LOSS: 0.0245 | ACC 0.9931
TEST: BATCH 0010/0157 | LOSS: 0.0326 | ACC 0.9922
TEST: BATCH 0011/0157 | LOSS: 0.0333 | ACC 0.9915
TEST: BATCH 0012/0157 | LOSS: 0.0369 | ACC 0.9883
TEST: BATCH 0013/0157 | LOSS: 0.0346 | ACC 0.9892
TEST: BATCH 0014/0157 | LOSS: 0.0350 | ACC 0.9888
TEST: BATCH 0015/0157 | LOSS: 0.0436 | ACC 0.9865
TEST: BATCH 0016/0157 | LOSS: 0.0475 | ACC 0.9854
TEST: BATCH 0017/0157 | LOSS: 0.0469 | ACC 0.9853
TEST: BATCH 0018/0157 | LOSS: 0.0470 | ACC 0.9844
TEST: BATCH 0019/0157 | LOSS: 0.0457 | ACC 0.9852
TEST: BATCH 0020/0157 | LOSS: 0.0522 | ACC 0.9820
